In [ ]:
 from google.colab import userdata
import os

In [ ]:
os.environ["KAGGLE_USERNAME"] = "pramothkumarm"
os.environ["KAGGLE_KEY"] = "ce704a0afd23bdba231fa81e0aa2fbbd"

In [ ]:
!kaggle datasets download -d pranavraikokte/covid19-image-dataset

covid19-image-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
!unzip "/content/covid19-image-dataset.zip"

Archive:  /content/covid19-image-dataset.zip
replace Covid19-dataset/test/Covid/0100.jpeg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
train_dir = '/content/Covid19-dataset/train'
test_dir = '/content/Covid19-dataset/test'


In [ ]:
import tensorflow as tf


In [ ]:
# Load training data
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir,
    image_size=(224, 224),
    batch_size=32,
    label_mode='int'
)


Found 251 files belonging to 3 classes.


In [ ]:
# Load testing data
test_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    test_dir,
    image_size=(224, 224),
    batch_size=32,
    label_mode='int'
)
num_classes = 15


Found 66 files belonging to 3 classes.


In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D,Dropout

In [ ]:
#LeNet

lenet_model = tf.keras.Sequential([
  tf.keras.layers.Conv2D(6, 5, activation='relu',input_shape=(224, 224, 3)),
  tf.keras.layers.AveragePooling2D(),
  tf.keras.layers.Conv2D(16, 5, activation='relu'),
  tf.keras.layers.AveragePooling2D(),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(120, activation='relu'),
  tf.keras.layers.Dense(84, activation='relu'),
  tf.keras.layers.Dense(3, activation='softmax')
])

In [ ]:
lenet_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

lenet_model.summary()

history = lenet_model.fit(train_dataset, epochs=10, validation_data=test_dataset)



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 220, 220, 6)       456       
                                                                 
 average_pooling2d (Average  (None, 110, 110, 6)       0         
 Pooling2D)                                                      
                                                                 
 conv2d_1 (Conv2D)           (None, 106, 106, 16)      2416      
                                                                 
 average_pooling2d_1 (Avera  (None, 53, 53, 16)        0         
 gePooling2D)                                                    
                                                                 
 flatten (Flatten)           (None, 44944)             0         
                                                                 
 dense (Dense)               (None, 120)               5

In [ ]:
loss, accuracy = lenet_model.evaluate(test_dataset)
print('Test loss:', loss)
print('Test accuracy:', accuracy)

3/3 [==============================] - 0s 11ms/step - loss: 1.0710 - accuracy: 0.7576
Test loss: 1.071000099182129
Test accuracy: 0.7575757503509521


In [ ]:
#AlexNet

alexnet = tf.keras.Sequential([
    tf.keras.layers.Conv2D(filters=96, kernel_size=(11,11), strides=(4,4), activation='relu', input_shape=(224,224,3)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
    tf.keras.layers.Conv2D(filters=256, kernel_size=(5,5), strides=(1,1), activation='relu', padding="same"),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
    tf.keras.layers.Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(4096, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(4096, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(3, activation='softmax')
])


In [ ]:
alexnet.compile(loss='sparse_categorical_crossentropy', optimizer=tf.optimizers.SGD(learning_rate=0.001), metrics=['accuracy'])
alexnet.summary()

history = alexnet.fit(train_dataset, epochs=10, validation_data=test_dataset)

loss, accuracy = alexnet.evaluate(test_dataset)
print('Test loss:', loss)
print('Test accuracy:', accuracy)


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 54, 54, 96)        34944     
                                                                 
 batch_normalization (Batch  (None, 54, 54, 96)        384       
 Normalization)                                                  
                                                                 
 max_pooling2d (MaxPooling2  (None, 26, 26, 96)        0         
 D)                                                              
                                                                 
 conv2d_3 (Conv2D)           (None, 26, 26, 256)       614656    
                                                                 
 batch_normalization_1 (Bat  (None, 26, 26, 256)       1024      
 chNormalization)                                                
                                                      

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D,Dropout


# Load the InceptionV3 (GoogLeNet) model with pre-trained weights
base_model = InceptionV3(weights='imagenet', include_top=False)

base_model.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(3, activation='softmax')(x)

model = tf.keras.Model(inputs=base_model.input, outputs=predictions)

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

history = model.fit(train_dataset, epochs=10, validation_data=test_dataset)

loss, accuracy = model.evaluate(test_dataset)
print('Test loss:', loss)
print('Test accuracy:', accuracy)

Epoch 1/10
8/8 [==============================] - 17s 924ms/step - loss: 35.1681 - accuracy: 0.4741 - val_loss: 15.0497 - val_accuracy: 0.6970
Epoch 2/10
8/8 [==============================] - 4s 152ms/step - loss: 14.0821 - accuracy: 0.6135 - val_loss: 3.9582 - val_accuracy: 0.7576
Epoch 3/10
8/8 [==============================] - 5s 151ms/step - loss: 5.9115 - accuracy: 0.7649 - val_loss: 2.3004 - val_accuracy: 0.9091
Epoch 4/10
8/8 [==============================] - 4s 151ms/step - loss: 3.7929 - accuracy: 0.7769 - val_loss: 1.1603 - val_accuracy: 0.8636
Epoch 5/10
8/8 [==============================] - 4s 154ms/step - loss: 1.4421 - accuracy: 0.8486 - val_loss: 1.0163 - val_accuracy: 0.8939
Epoch 6/10
8/8 [==============================] - 5s 153ms/step - loss: 1.0888 - accuracy: 0.8566 - val_loss: 0.4097 - val_accuracy: 0.8939
Epoch 7/10
8/8 [==============================] - 4s 150ms/step - loss: 0.4745 - accuracy: 0.8526 - val_loss: 0.4032 - val_accuracy: 0.7879
Epoch 8/10
8/8 [

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D,Dropout

base_model = ResNet50(weights='imagenet', include_top=False)

base_model.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)

predictions = Dense(3, activation='softmax')(x)

model = tf.keras.Model(inputs=base_model.input, outputs=predictions)

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

history = model.fit(train_dataset, epochs=10, validation_data=test_dataset)

loss, accuracy = model.evaluate(test_dataset)
print('Test loss:', loss)
print('Test accuracy:', accuracy)



Epoch 1/10
8/8 [==============================] - 13s 684ms/step - loss: 1.3521 - accuracy: 0.5578 - val_loss: 0.5255 - val_accuracy: 0.7879
Epoch 2/10
8/8 [==============================] - 4s 186ms/step - loss: 0.4935 - accuracy: 0.7968 - val_loss: 0.3935 - val_accuracy: 0.8182
Epoch 3/10
8/8 [==============================] - 5s 179ms/step - loss: 0.3841 - accuracy: 0.8446 - val_loss: 0.3820 - val_accuracy: 0.7879
Epoch 4/10
8/8 [==============================] - 4s 190ms/step - loss: 0.2326 - accuracy: 0.9163 - val_loss: 0.2651 - val_accuracy: 0.9545
Epoch 5/10
8/8 [==============================] - 5s 284ms/step - loss: 0.2228 - accuracy: 0.9163 - val_loss: 0.2410 - val_accuracy: 0.9091
Epoch 6/10
8/8 [==============================] - 4s 181ms/step - loss: 0.1642 - accuracy: 0.9442 - val_loss: 0.1719 - val_accuracy: 0.9697
Epoch 7/10
8/8 [==============================] - 4s 177ms/step - loss: 0.1274 - accuracy: 0.9681 - val_loss: 0.1689 - val_accuracy: 0.9545
Epoch 8/10
8/8 [===

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D,Dropout

base_model = VGG16(weights='imagenet', include_top=False)

base_model.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(3, activation='softmax')(x)

model = tf.keras.Model(inputs=base_model.input, outputs=predictions)

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

history = model.fit(train_dataset, epochs=10, validation_data=test_dataset)

loss, accuracy = model.evaluate(test_dataset)
print('Test loss:', loss)
print('Test accuracy:', accuracy)

Epoch 1/10
8/8 [==============================] - 20s 1s/step - loss: 4.2802 - accuracy: 0.4980 - val_loss: 1.1245 - val_accuracy: 0.7273
Epoch 2/10
8/8 [==============================] - 5s 315ms/step - loss: 1.3685 - accuracy: 0.7610 - val_loss: 0.5595 - val_accuracy: 0.8333
Epoch 3/10
8/8 [==============================] - 5s 217ms/step - loss: 0.7168 - accuracy: 0.8287 - val_loss: 0.5247 - val_accuracy: 0.8182
Epoch 4/10
8/8 [==============================] - 6s 217ms/step - loss: 0.4357 - accuracy: 0.8845 - val_loss: 0.3445 - val_accuracy: 0.8939
Epoch 5/10
8/8 [==============================] - 5s 215ms/step - loss: 0.5266 - accuracy: 0.8964 - val_loss: 0.3300 - val_accuracy: 0.8788
Epoch 6/10
8/8 [==============================] - 5s 312ms/step - loss: 0.1782 - accuracy: 0.9402 - val_loss: 0.3775 - val_accuracy: 0.8636
Epoch 7/10
8/8 [==============================] - 4s 216ms/step - loss: 0.2490 - accuracy: 0.9243 - val_loss: 0.2042 - val_accuracy: 0.9091
Epoch 8/10
8/8 [======